# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [7]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [8]:
df = pd.read_csv('previsao_de_renda.csv')
df.head(2)

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [40]:
#Verificando a melhor opção para casela
df['estado_civil'].value_counts()

Casado      10534
Solteiro     1798
União        1078
Separado      879
Viúvo         711
Name: estado_civil, dtype: int64

In [41]:
from patsy import dmatrix
dmatrix("C(estado_civil)", df)

DesignMatrix with shape (15000, 5)
  Columns:
    ['Intercept',
     'C(estado_civil)[T.Separado]',
     'C(estado_civil)[T.Solteiro]',
     'C(estado_civil)[T.União]',
     'C(estado_civil)[T.Viúvo]']
  Terms:
    'Intercept' (column 0), 'C(estado_civil)' (columns 1:5)
  (to view full data, use np.asarray(this_obj))

In [38]:
#1)
y, x = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil) + C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia', data = df)
x

DesignMatrix with shape (12427, 25)
  Columns:
    ['Intercept',
     'sexo[T.M]',
     'posse_de_veiculo[T.True]',
     'posse_de_imovel[T.True]',
     'C(tipo_renda)[T.Bolsista]',
     'C(tipo_renda)[T.Empresário]',
     'C(tipo_renda)[T.Pensionista]',
     'C(tipo_renda)[T.Servidor público]',
     'C(educacao, Treatment(2))[T.Primário]',
     'C(educacao, Treatment(2))[T.Pós graduação]',
     'C(educacao, Treatment(2))[T.Superior completo]',
     'C(educacao, Treatment(2))[T.Superior incompleto]',
     'C(estado_civil)[T.Separado]',
     'C(estado_civil)[T.Solteiro]',
     'C(estado_civil)[T.União]',
     'C(estado_civil)[T.Viúvo]',
     'C(tipo_residencia)[T.Casa]',
     'C(tipo_residencia)[T.Com os pais]',
     'C(tipo_residencia)[T.Comunitário]',
     'C(tipo_residencia)[T.Estúdio]',
     'C(tipo_residencia)[T.Governamental]',
     'qtd_filhos',
     'idade',
     'tempo_emprego',
     'qt_pessoas_residencia']
  Terms:
    'Intercept' (column 0)
    'sexo' (column 1)
    'posse_d

In [42]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Sun, 13 Mar 2022   Prob (F-statistic):               0.00
Time:                        14:49:40   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            6.5706      0.225     29.256      0.000       6.130       7.011
sexo[T.M]                                            0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]                             0.0441      0.014      3.119      0.002       0.016       0.072
posse_de_imovel[T.True]                              0.0829      0.014      5.926      0.000       0.055       0.110
C(tipo_renda)[T.Bolsista]                            0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda)[T.Empresário]                          0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda)[T.Pensionista]                        -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda)[T.Servidor público]                    0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment(2))[T.Primário]                0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment(2))[T.Pós graduação]           0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment(2))[T.Superior completo]       0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil)[T.Separado]                          0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil)[T.Solteiro]                          0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil)[T.União]                            -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil)[T.Viúvo]                             0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia)[T.Casa]                          -0.0442      0.054     -0.815      0.415      -0.150       0.062
C(tipo_residencia)[T.Com os pais]                   -0.0258      0.060     -0.427      0.670      -0.144       0.093
C(tipo_residencia)[T.Comunitário]                   -0.1196      0.105     -1.135      0.256      -0.326       0.087
C(tipo_residencia)[T.Estúdio]                        0.0665      0.099      0.670      0.503      -0.128       0.261
C(tipo_residencia)[T.Governamental]                 -0.0571      0.066     -0.865      0.387      -0.186       0.072
qtd_filhos                                          -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                                0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego                            

In [17]:
df['tipo_renda'].value_counts()

Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: tipo_renda, dtype: int64

#### Insight
<p>Muitas variáveis parecem fazer bastante sentido por seu p-value, porém o modelo é muito complexto diante do valor de AIC e BIC.</p>

In [43]:
#2) Removendo a Variável tipo_residencia
y, x = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao) + C(estado_civil) + idade + tempo_emprego + qt_pessoas_residencia', data = df)

In [44]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     382.4
Date:                Sun, 13 Mar 2022   Prob (F-statistic):               0.00
Time:                        14:49:59   Log-Likelihood:                -13574.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12408   BIC:                         2.733e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              6.5602      0.229     28.615      0.000       6.111       7.010
sexo[T.M]                              0.8036      0.014     57.886      0.000       0.776       0.831
posse_de_imovel[T.True]                0.0830      0.014      6.048      0.000       0.056       0.110
C(tipo_renda)[T.Bolsista]              0.2061      0.241      0.854      0.393      -0.267       0.679
C(tipo_renda)[T.Empresário]            0.1562      0.015     10.475      0.000       0.127       0.185
C(tipo_renda)[T.Pensionista]          -0.3113      0.241     -1.290      0.197      -0.784       0.162
C(tipo_renda)[T.Servidor público]      0.0584      0.022      2.627      0.009       0.015       0.102
C(educacao)[T.Pós graduação]           0.1225      0.159      0.771      0.441      -0.189       0.434
C(educacao)[T.Secundário]             -0.0093      0.072     -0.129      0.898      -0.150       0.132
C(educacao)[T.Superior completo]       0.1018      0.072      1.412      0.158      -0.040       0.243
C(educacao)[T.Superior incompleto]    -0.0380      0.078     -0.489      0.625      -0.190       0.114
C(estado_civil)[T.Separado]            0.3188      0.111      2.866      0.004       0.101       0.537
C(estado_civil)[T.Solteiro]            0.2616      0.109      2.402      0.016       0.048       0.475
C(estado_civil)[T.União]              -0.0384      0.025     -1.530      0.126      -0.088       0.011
C(estado_civil)[T.Viúvo]               0.3634      0.116      3.143      0.002       0.137       0.590
qtd_filhos                            -0.2636      0.108     -2.441      0.015      -0.475      -0.052
idade                                  0.0051      0.001      6.449      0.000       0.004       0.007
tempo_emprego                          0.0617      0.001     59.431      0.000       0.060       0.064
qt_pessoas_residencia                  0.2912      0.108      2.702      0.007       0.080       0.502
==============================================================================
Omnibus:                        0.793   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.673   Jarque-Bera (JB):                0.782
Skew:                           0.019   Prob(JB):                        0.676
Kurtosis:                       3.006   Cond. No.                     2.16e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.16e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [53]:
#Verificando a complexidade do modelo 1
reg = smf.ols('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil) + idade + tempo_emprego + qt_pessoas_residencia', data = df).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Sun, 13 Mar 2022   Prob (F-statistic):               0.00
Time:                        14:54:36   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            6.5315      0.218     29.894      0.000       6.103       6.960
sexo[T.M]                                            0.7893      0.015     53.999      0.000       0.761       0.818
posse_de_veiculo[T.True]                             0.0442      0.014      3.125      0.002       0.016       0.072
posse_de_imovel[T.True]                              0.0819      0.014      5.966      0.000       0.055       0.109
C(tipo_renda)[T.Bolsista]                            0.2219      0.241      0.920      0.357      -0.251       0.695
C(tipo_renda)[T.Empresário]                          0.1556      0.015     10.433      0.000       0.126       0.185
C(tipo_renda)[T.Pensionista]                        -0.3116      0.241     -1.292      0.196      -0.784       0.161
C(tipo_renda)[T.Servidor público]                    0.0589      0.022      2.653      0.008       0.015       0.102
C(educacao, Treatment(2))[T.Primário]                0.0097      0.072      0.135      0.893      -0.131       0.150
C(educacao, Treatment(2))[T.Pós graduação]           0.1200      0.142      0.845      0.398      -0.159       0.399
C(educacao, Treatment(2))[T.Superior completo]       0.1077      0.014      7.761      0.000       0.080       0.135
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0288      0.032     -0.893      0.372      -0.092       0.034
C(estado_civil)[T.Separado]                          0.3272      0.111      2.942      0.003       0.109       0.545
C(estado_civil)[T.Solteiro]                          0.2683      0.109      2.464      0.014       0.055       0.482
C(estado_civil)[T.União]                            -0.0329      0.025     -1.310      0.190      -0.082       0.016
C(estado_civil)[T.Viúvo]                             0.3678      0.116      3.182      0.001       0.141       0.594
qtd_filhos                                          -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                                0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                                        0.0617      0.001     59.485      0.000       0.060       0.064
qt_pessoas_residencia                                0.2923      0.108      2.713      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.825   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.662   Jarque-Bera (JB):                0.809
Skew:                           0.019   Prob(JB):                        0.667
Kurtosis:                       3.009   Cond. No.                     2.13e+

In [54]:
# Verificando a complexidade do modelo 2
reg = smf.ols('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil) + idade + tempo_emprego + qt_pessoas_residencia', data = df).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Sun, 13 Mar 2022   Prob (F-statistic):               0.00
Time:                        14:54:39   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            6.5315      0.218     29.894      0.000       6.103       6.960
sexo[T.M]                                            0.7893      0.015     53.999      0.000       0.761       0.818
posse_de_veiculo[T.True]                             0.0442      0.014      3.125      0.002       0.016       0.072
posse_de_imovel[T.True]                              0.0819      0.014      5.966      0.000       0.055       0.109
C(tipo_renda)[T.Bolsista]                            0.2219      0.241      0.920      0.357      -0.251       0.695
C(tipo_renda)[T.Empresário]                          0.1556      0.015     10.433      0.000       0.126       0.185
C(tipo_renda)[T.Pensionista]                        -0.3116      0.241     -1.292      0.196      -0.784       0.161
C(tipo_renda)[T.Servidor público]                    0.0589      0.022      2.653      0.008       0.015       0.102
C(educacao, Treatment(2))[T.Primário]                0.0097      0.072      0.135      0.893      -0.131       0.150
C(educacao, Treatment(2))[T.Pós graduação]           0.1200      0.142      0.845      0.398      -0.159       0.399
C(educacao, Treatment(2))[T.Superior completo]       0.1077      0.014      7.761      0.000       0.080       0.135
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0288      0.032     -0.893      0.372      -0.092       0.034
C(estado_civil)[T.Separado]                          0.3272      0.111      2.942      0.003       0.109       0.545
C(estado_civil)[T.Solteiro]                          0.2683      0.109      2.464      0.014       0.055       0.482
C(estado_civil)[T.União]                            -0.0329      0.025     -1.310      0.190      -0.082       0.016
C(estado_civil)[T.Viúvo]                             0.3678      0.116      3.182      0.001       0.141       0.594
qtd_filhos                                          -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                                0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                                        0.0617      0.001     59.485      0.000       0.060       0.064
qt_pessoas_residencia                                0.2923      0.108      2.713      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.825   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.662   Jarque-Bera (JB):                0.809
Skew:                           0.019   Prob(JB):                        0.667
Kurtosis:                       3.009   Cond. No.                     2.13e+

#### Insight
<p> O modelo apresentou praticamente o mesmo resultado do primeiro, sendo necessário a remoção de mais variáveis que são insignificantes.</p>

In [51]:
#3)
y,x = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_imovel + C(tipo_renda) + idade + tempo_emprego', data = df)

In [52]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     841.8
Date:                Sun, 13 Mar 2022   Prob (F-statistic):               0.00
Time:                        14:51:36   Log-Likelihood:                -13624.
No. Observations:               12427   AIC:                         2.727e+04
Df Residuals:                   12418   BIC:                         2.733e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             7.2089      0.032    227.180      0.000       7.147       7.271
sexo[T.M]                             0.7991      0.014     58.106      0.000       0.772       0.826
posse_de_imovel[T.True]               0.0881      0.014      6.420      0.000       0.061       0.115
C(tipo_renda)[T.Bolsista]             0.2724      0.242      1.126      0.260      -0.202       0.746
C(tipo_renda)[T.Empresário]           0.1614      0.015     10.861      0.000       0.132       0.190
C(tipo_renda)[T.Pensionista]         -0.2688      0.242     -1.112      0.266      -0.743       0.205
C(tipo_renda)[T.Servidor público]     0.0778      0.022      3.510      0.000       0.034       0.121
idade                                 0.0044      0.001      5.963      0.000       0.003       0.006
tempo_emprego                         0.0617      0.001     59.335      0.000       0.060       0.064
==============================================================================
Omnibus:                        0.973   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.615   Jarque-Bera (JB):                0.945
Skew:                           0.019   Prob(JB):                        0.623
Kurtosis:                       3.020   Cond. No.                     1.58e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<p>O modelo com mais variáveis, o primeiro, ficou com R² maior que o segundo, e ambos os modelos ficaram complexos, mesmo mudando a casela nas variáveis, não surgiu efeito.</p>